![](https://www.qads.com.br/img/QADS.png)

## Integrando Python e SQL no Azure Data Studio

Nesse caderno mostraremos como extrair dados do banco SQL Server diretamente para coleções no Python.

Antes de começar é preciso configurar o driver ODBC da sua máquina e criar uma DSN (Data Source Name) de usuário. Para fazer isso siga:

- Windows Start (pesquisa genérica do Windows : "Digite aqui para pesquisar" ) > odbc > ODBC Data Sources (ou Fontes de dados ODBC) para abrir o aplicativo "Administrador de fontes de dados ODBC".

- No aplicativo crie uma DSN de usuário: Adicionar > ODBC Driver 17 for SQL Server > escolha um nome > escolha uma descrição > indique o servidor SQL Server > Com Autenticação Integrada do Windows >  avance até concluir > teste a conexão.

## Pyodbc

Verifique se o pyodbc está instalado no seu servidor (Manage Packages). Se não estiver adicione-o seguindo os passos na aba "Add new" do aplicativo Manage Packages.


In [1]:
import pyodbc
# conectando com o banco de dados
cnxn = pyodbc.connect("DSN=sqlserver") # use aqui o DSN criado no aplicativo de administração do ODBC
cursor = cnxn.cursor()

O cursor permite a transferência de dados do banco para o python. O resultado de uma consulta, passada como parâmetro do comando "execute" poderá ser transferida para coleções de dados.

In [11]:
cursor.execute("select TOP 30 * FROM [AdventureWorks2017].[Person].[Person]")
# lendo o cursor linha a linha
row = cursor.fetchone()
while row:
    print (str(row[4]) + " " + str(row[6]))
    row = cursor.fetchone()

Ken Sánchez
Terri Duffy
Roberto Tamburello
Rob Walters
Gail Erickson
Jossef Goldberg
Dylan Miller
Diane Margheim
Gigi Matthew
Michael Raheem
Ovidiu Cracium
Thierry D'Hers
Janice Galvin
Michael Sullivan
Sharon Salavaria
David Bradley
Kevin Brown
John Wood
Mary Dempsey
Wanida Benshoof
Terry Eminhizer
Sariya Harnpadoungsataya
Mary Gibson
Jill Williams
James Hamilton
Peter Krebs
Jo Brown
Guy Gilbert
Mark McArthur
Britta Simon


Após o loop de fetchs o cursos "esvazia". Para usar os dados novamente ou a consulta deve ser reexecutada ou os dados transferidos para uma coleção do Python.

O fetchall no cursor retorna uma lista de listas.

In [13]:
cursor.execute("select TOP 10 * FROM [AdventureWorks2017].[Person].[Person]")
# transferindo o cursor para uma lista de listas
tabela = cursor.fetchall()
for linha in tabela:
    print(linha, end='\n')

(1, 'EM', False, None, 'Ken', 'J', 'Sánchez', None, 0, None, '<IndividualSurvey xmlns="http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/IndividualSurvey"><TotalPurchaseYTD>0</TotalPurchaseYTD></IndividualSurvey>', '92C4279F-1207-48A3-8448-4636514EB7E2', datetime.datetime(2009, 1, 7, 0, 0))
(2, 'EM', False, None, 'Terri', 'Lee', 'Duffy', None, 1, None, '<IndividualSurvey xmlns="http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/IndividualSurvey"><TotalPurchaseYTD>0</TotalPurchaseYTD></IndividualSurvey>', 'D8763459-8AA8-47CC-AFF7-C9079AF79033', datetime.datetime(2008, 1, 24, 0, 0))
(3, 'EM', False, None, 'Roberto', None, 'Tamburello', None, 0, None, '<IndividualSurvey xmlns="http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/IndividualSurvey"><TotalPurchaseYTD>0</TotalPurchaseYTD></IndividualSurvey>', 'E1A2555E-0828-434B-A33B-6F38136A37DE', datetime.datetime(2007, 11, 4, 0, 0))
(4, 'EM', False, None, 'Rob', None, 'Walters', None, 0, None, '<Individua

In [14]:
tabela[0]


(1, 'EM', False, None, 'Ken', 'J', 'Sánchez', None, 0, None, '<IndividualSurvey xmlns="http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/IndividualSurvey"><TotalPurchaseYTD>0</TotalPurchaseYTD></IndividualSurvey>', '92C4279F-1207-48A3-8448-4636514EB7E2', datetime.datetime(2009, 1, 7, 0, 0))

Cada posição em uma dada linha da lista de listas corresponde a uma coluna do resultado da consulta, com o tipo de dado correspondente (o que é uma grande vantagem com relação à importação de flat files). Entretanto, o nome das colunas se perde na transferência. o acesso deve ser feito só por posição.

In [15]:
for i in range(len(tabela[0])):
    print(tabela[0][i],type(tabela[0][i]))

1 <class 'int'>
EM <class 'str'>
False <class 'bool'>
None <class 'NoneType'>
Ken <class 'str'>
J <class 'str'>
Sánchez <class 'str'>
None <class 'NoneType'>
0 <class 'int'>
None <class 'NoneType'>
<IndividualSurvey xmlns="http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/IndividualSurvey"><TotalPurchaseYTD>0</TotalPurchaseYTD></IndividualSurvey> <class 'str'>
92C4279F-1207-48A3-8448-4636514EB7E2 <class 'str'>
2009-01-07 00:00:00 <class 'datetime.datetime'>


In [16]:
type(tabela)


list

Consultas longas, que ocupam várias linha, podem ser preparadas usando o pacote textwrap e aspas triplas:

In [22]:
import textwrap
tsql =textwrap.dedent("""SELECT B.SalesPersonID
      ,C.JobTitle
      ,C.Gender
      ,B.[TerritoryID] 
      ,D.Name as NomeTerritorioVenda
      ,d.CountryRegionCode
      ,[SalesQuota]
      ,[Bonus]
      ,[CommissionPct]
      ,A.[SalesYTD]
      ,A.[SalesLastYear]
      ,B.OrderDate
      ,B.SubTotal
      ,B.TotalDue
      ,B.CustomerID
      ,B.SalesOrderID
      ,B.ShipDate
      ,B.Freight
  FROM [AdventureWorks2017].[Sales].[SalesPerson] A
  FULL OUTER JOIN [AdventureWorks2017].[Sales].SalesOrderHeader B on A.BusinessEntityID = B.SalesPersonID
  FULL OUTER JOIN [AdventureWorks2017].[HumanResources].Employee C ON C.BusinessEntityID = A.BusinessEntityID
  INNER JOIN [AdventureWorks2017].[Sales].SalesTerritory D ON D.TerritoryID = B.TerritoryID""")
cursor.execute(tsql) 
tabela = cursor.fetchall()
for i in range(10) :
 print(tabela[i], end='\n')

(279, 'Sales Representative', 'M', 5, 'Southeast', 'US', Decimal('300000.0000'), Decimal('6700.0000'), Decimal('0.0100'), Decimal('2315185.6110'), Decimal('1849640.9418'), datetime.datetime(2011, 5, 31, 0, 0), Decimal('20565.6206'), Decimal('23153.2339'), 29825, 43659, datetime.datetime(2011, 6, 7, 0, 0), Decimal('616.0984'))
(279, 'Sales Representative', 'M', 5, 'Southeast', 'US', Decimal('300000.0000'), Decimal('6700.0000'), Decimal('0.0100'), Decimal('2315185.6110'), Decimal('1849640.9418'), datetime.datetime(2011, 5, 31, 0, 0), Decimal('1294.2529'), Decimal('1457.3288'), 29672, 43660, datetime.datetime(2011, 6, 7, 0, 0), Decimal('38.8276'))
(282, 'Sales Representative', 'M', 6, 'Canada', 'CA', Decimal('250000.0000'), Decimal('5000.0000'), Decimal('0.0150'), Decimal('2604540.7172'), Decimal('2038234.6549'), datetime.datetime(2011, 5, 31, 0, 0), Decimal('32726.4786'), Decimal('36865.8012'), 29734, 43661, datetime.datetime(2011, 6, 7, 0, 0), Decimal('985.5530'))
(282, 'Sales Represent

# Exercício 1

Extraia do banco de dados o total de vendas em 2011 de cada vendededor que atue na França. Obtenha  Inspecione a coleção obtida.

## Convertendo os dados da consulta num Pandas Dataframe

Pode ser feito convertendo a lista em dataframe ou usando o pandas diretamente:

In [23]:
import pandas
tabelapandas = pandas.DataFrame(tabela)

In [24]:
tabelapandas.head()


,0
0,"[279, Sales Representative, M, 5, Southeast, U..."
1,"[279, Sales Representative, M, 5, Southeast, U..."
2,"[282, Sales Representative, M, 6, Canada, CA, ..."
3,"[282, Sales Representative, M, 6, Canada, CA, ..."
4,"[276, Sales Representative, F, 4, Southwest, U..."


Entretanto, como a conversão em dataframe foi feita a partir da lista de listas o nome dos campos (colunas) se perdeu.

In [25]:
tabelapandas.columns

RangeIndex(start=0, stop=1, step=1)

## Carregando dados diretamente do banco SQL para um dataframe

Usando o método read_sql do Pandas a carga dos dados preserva não só os tipos de dados mas também os nomes de colunas.

In [26]:
sql = "select * FROM [AdventureWorks2017].[Person].[Person]"
data = pandas.read_sql(sql,cnxn)

In [27]:
data.shape

(19972, 13)

In [28]:
data.describe()

,BusinessEntityID,EmailPromotion
count,19972.000000,19972.000000
mean,10763.079411,0.630082
std,5814.133272,0.781433
min,1.000000,0.000000
25%,5798.750000,0.000000
50%,10791.500000,0.000000
75%,15784.250000,1.000000
max,20777.000000,2.000000


In [29]:
data.columns

Index(['BusinessEntityID', 'PersonType', 'NameStyle', 'Title', 'FirstName',
       'MiddleName', 'LastName', 'Suffix', 'EmailPromotion',
       'AdditionalContactInfo', 'Demographics', 'rowguid', 'ModifiedDate'],
      dtype='object')

Fazendo seleções nos dados por meio de slices no dataframe:

In [31]:
data[data['PersonType']=='SC']

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
290,291,SC,False,Mr.,Gustavo,None,Achong,None,2,"<AdditionalContactInfo xmlns=""http://schemas.m...","<IndividualSurvey xmlns=""http://schemas.micros...",D4C132D3-FCB5-4231-9DD5-888A54BEC693,2015-04-15 16:33:33.060
291,293,SC,False,Ms.,Catherine,R.,Abel,None,1,"<AdditionalContactInfo xmlns=""http://schemas.m...","<IndividualSurvey xmlns=""http://schemas.micros...",D54E0552-C226-4C22-AF3B-762CA854CDD3,2015-04-15 16:33:33.077
292,295,SC,False,Ms.,Kim,None,Abercrombie,None,0,"<AdditionalContactInfo xmlns=""http://schemas.m...","<IndividualSurvey xmlns=""http://schemas.micros...",F7CBDB48-0B44-470E-9F37-7060446FBFB9,2015-04-15 16:33:33.077
293,297,SC,False,Sr.,Humberto,None,Acevedo,None,2,"<AdditionalContactInfo xmlns=""http://schemas.m...","<IndividualSurvey xmlns=""http://schemas.micros...",5A41D336-84CF-44D7-B12B-83B64B511F7E,2015-04-15 16:33:33.090
294,299,SC,False,Sra.,Pilar,None,Ackerman,None,0,"<AdditionalContactInfo xmlns=""http://schemas.m...","<IndividualSurvey xmlns=""http://schemas.micros...",DF1FB8AB-2323-4330-9AB8-54E13CE6D8F9,2015-04-15 16:33:33.090
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,2033,SC,False,Mr.,Sam,L.,Wheeler,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",232712CE-3D4C-473C-9B90-21B13E67DE4A,2012-07-31 00:00:00.000
1229,2034,SC,False,Ms.,Wendy,None,Wheeler,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",AB3887D3-12B0-4416-85FE-92CEB6EE8A0D,2011-08-01 00:00:00.000
1230,2035,SC,False,Ms.,Vivian,M.,Whipple,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",FA45D72B-17EE-494F-AB7B-30D5CAA6B13C,2011-07-01 00:00:00.000
1231,2036,SC,False,Ms.,Cynthia,J.,White,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",4273A8E5-96F1-4839-8294-F5C97742E3C0,2012-06-30 00:00:00.000


In [32]:
# versão pandas de uma query com filtro, projeção e função count distinct do sql
data[data['PersonType']=='SC']['Title'].nunique()

4

# Exercício 2

Repita o exercício 1 trazendo do banco a junção completa das tabelas envolvidas para um dataframe Pandas e realizando os filtros pedidos no dataframe.